# Glue Elastic Views Overview

Using AWS Glue EV Console

<img src="img/img2.png">

Replicating data across multiple AWS data stores using AWS Glue Elastic Views involves the following steps:

<img src="img/img1.png">

# Step 1: Create a `DynamoDB` table (`SOURCE`)

In [1]:
%%bash 

aws dynamodb create-table \
    --table-name favoritemovies \
    --attribute-definitions \
        AttributeName=title,AttributeType=S \
        AttributeName=release_year,AttributeType=N \
    --key-schema \
        AttributeName=title,KeyType=HASH \
        AttributeName=release_year,KeyType=RANGE \
    --provisioned-throughput \
            ReadCapacityUnits=10,WriteCapacityUnits=5 | jq . 

{
  "TableDescription": {
    "AttributeDefinitions": [
      {
        "AttributeName": "release_year",
        "AttributeType": "N"
      },
      {
        "AttributeName": "title",
        "AttributeType": "S"
      }
    ],
    "TableName": "favoritemovies",
    "KeySchema": [
      {
        "AttributeName": "title",
        "KeyType": "HASH"
      },
      {
        "AttributeName": "release_year",
        "KeyType": "RANGE"
      }
    ],
    "TableStatus": "CREATING",
    "CreationDateTime": "2021-06-14T11:59:01.639000+05:30",
    "ProvisionedThroughput": {
      "NumberOfDecreasesToday": 0,
      "ReadCapacityUnits": 10,
      "WriteCapacityUnits": 5
    },
    "TableSizeBytes": 0,
    "ItemCount": 0,
    "TableArn": "arn:aws:dynamodb:us-east-1:507922848584:table/favoritemovies",
    "TableId": "e99101a6-9b18-4692-ad98-feb9e6e4bbcb"
  }
}


### Use describe-table to verify that `DynamoDB` finished creating the `favoritemovies` table and to find the `ARN` of your table

In [2]:
!aws dynamodb describe-table --table-name favoritemovies | grep -e TableStatus -e TableArn


        "TableStatus": "ACTIVE",
        "TableArn": "arn:aws:dynamodb:us-east-1:507922848584:table/favoritemovies",


In [3]:
dynamoDB_ARN = "arn:aws:dynamodb:us-east-1:507922848584:table/favoritemovies"

### Put `items` in the Table: 

In [4]:
%%bash

# Item 1
aws dynamodb put-item \
--table-name favoritemovies  \
--item \
    '{"title": {"S": "Inception"}, "release_year": {"N": "2010"}}'

# Item 2
aws dynamodb put-item \
    --table-name favoritemovies \
    --item \
    '{"title": {"S": "Moonlight"}, "release_year": {"N": "2016"}}'

#Item 3
aws dynamodb put-item \
    --table-name favoritemovies \
    --item \
    '{"title": {"S": "The Dark Knight"}, "release_year": {"N": "2008"}}'

### Run `scan` to check that the data is added to the table 

In [5]:
!aws dynamodb scan --table-name favoritemovies |jq . 


{
  "Items": [
    {
      "release_year": {
        "N": "2008"
      },
      "title": {
        "S": "The Dark Knight"
      }
    },
    {
      "release_year": {
        "N": "2010"
      },
      "title": {
        "S": "Inception"
      }
    },
    {
      "release_year": {
        "N": "2016"
      },
      "title": {
        "S": "Moonlight"
      }
    }
  ],
  "Count": 3,
  "ScannedCount": 3,
  "ConsumedCapacity": null
}


<img src="img/img3.png">

# Step 2: Create an Elastic View(Table and View)

### Create an Elastic Views table

In [6]:
!aws elasticviews create-external-datastore \
    --external-datastore-identifier {dynamoDB_ARN} \
    --datastore-name evfavoritemovies --datastore-type DYNAMODB | jq . 

{
  "datastore": {
    "externalDatastoreIdentifier": "arn:aws:dynamodb:us-east-1:507922848584:table/favoritemovies",
    "datastoreName": "evfavoritemovies",
    "datastoreType": "DYNAMODB",
    "datastoreStatus": "CREATING",
    "datastoreArn": "arn:aws:elasticviews:us-east-1:507922848584:datastore/evfavoritemovies-7ISCuEHelKB42cQUCEDpKp",
    "createdTime": "2021-06-14T12:01:00.237000+05:30",
    "lastUpdatedTime": "2021-06-14T12:01:00.237000+05:30"
  }
}


In [7]:
!aws elasticviews list-sources | jq ".sources[-1]"  # Copy the sourceArn

{
  "datastoreName": "evfavoritemovies",
  "sourceNamespace": "evfavoritemovies.favoritemovies",
  "sourceType": "DYNAMODB",
  "datastoreArn": "arn:aws:elasticviews:us-east-1:507922848584:datastore/evfavoritemovies-7ISCuEHelKB42cQUCEDpKp",
  "sourceArn": "arn:aws:elasticviews:us-east-1:507922848584:source/favoritemovies-1yQgYQbwge35SHdA1yrzTq",
  "sourceSchema": {
    "attributeSchemaList": [
      {
        "attributeName": "release_year",
        "typeDefinition": {
          "typeName": "ddb:number",
          "typeConstraints": [
            {
              "constraintName": "Precision",
              "constraintValue": "[1, 38]"
            },
            {
              "constraintName": "ValidValues",
              "constraintValue": "[-9.9999999999999999999999999999999999999E+125, 9.9999999999999999999999999999999999999E+125]"
            },
            {
              "constraintName": "Required",
              "constraintValue": "ConstraintBoolean(value=true)"
            }
 

In [8]:
sourceArn = "arn:aws:elasticviews:us-east-1:507922848584:source/favoritemovies-1yQgYQbwge35SHdA1yrzTq"

### Activate the Elastic Views table

In [9]:
!aws elasticviews start-source \
    --source-identifier {sourceArn} | jq . 

{
  "source": {
    "datastoreName": "evfavoritemovies",
    "sourceNamespace": "evfavoritemovies.favoritemovies",
    "sourceType": "DYNAMODB",
    "datastoreArn": "arn:aws:elasticviews:us-east-1:507922848584:datastore/evfavoritemovies-7ISCuEHelKB42cQUCEDpKp",
    "sourceArn": "arn:aws:elasticviews:us-east-1:507922848584:source/favoritemovies-1yQgYQbwge35SHdA1yrzTq",
    "sourceSchema": {
      "attributeSchemaList": [
        {
          "attributeName": "release_year",
          "typeDefinition": {
            "typeName": "ddb:number",
            "typeConstraints": [
              {
                "constraintName": "Precision",
                "constraintValue": "[1, 38]"
              },
              {
                "constraintName": "ValidValues",
                "constraintValue": "[-9.9999999999999999999999999999999999999E+125, 9.9999999999999999999999999999999999999E+125]"
              },
              {
                "constraintName": "Required",
                "const

### Check the status of the Elastic Views table

In [13]:
!aws elasticviews describe-source \
    --source-identifier {sourceArn} | jq . 

{
  "source": {
    "datastoreName": "evfavoritemovies",
    "sourceNamespace": "evfavoritemovies.favoritemovies",
    "sourceType": "DYNAMODB",
    "datastoreArn": "arn:aws:elasticviews:us-east-1:507922848584:datastore/evfavoritemovies-7ISCuEHelKB42cQUCEDpKp",
    "sourceArn": "arn:aws:elasticviews:us-east-1:507922848584:source/favoritemovies-1yQgYQbwge35SHdA1yrzTq",
    "sourceSchema": {
      "attributeSchemaList": [
        {
          "attributeName": "release_year",
          "typeDefinition": {
            "typeName": "ddb:number",
            "typeConstraints": [
              {
                "constraintName": "Precision",
                "constraintValue": "[1, 38]"
              },
              {
                "constraintName": "ValidValues",
                "constraintValue": "[-9.9999999999999999999999999999999999999E+125, 9.9999999999999999999999999999999999999E+125]"
              },
              {
                "constraintName": "Required",
                "const

<img src="img/img4.png">

### Create an Elastic Views view

In [14]:
!aws elasticviews create-view \
        --view-definition 'SELECT CAST(release_year AS int4), title FROM evfavoritemovies.favoritemovies' \
        --provenances "release_year" "title" \
        --view-name favoritemovies-view | jq . 

{
  "view": {
    "viewName": "favoritemovies-view",
    "viewArn": "arn:aws:elasticviews:us-east-1:507922848584:view/favoritemovies-view-Gzwyrxov5Xk4aAY7Gyl3pu",
    "viewDefinition": "SELECT CAST(release_year AS int4), title FROM evfavoritemovies.favoritemovies",
    "provenances": [
      "release_year",
      "title"
    ],
    "viewStatus": "INACTIVE",
    "sourceOrViewArns": [
      "arn:aws:elasticviews:us-east-1:507922848584:source/favoritemovies-1yQgYQbwge35SHdA1yrzTq"
    ],
    "viewSchema": {
      "attributeSchemaList": [
        {
          "attributeName": "release_year",
          "typeDefinition": {
            "typeName": "int",
            "typeConstraints": [
              {
                "constraintName": "ValidValues",
                "constraintValue": "[-2147483648, 2147483647]"
              }
            ]
          }
        },
        {
          "attributeName": "title",
          "typeDefinition": {
            "typeName": "ddb:string",
            "type

In [15]:
viewArn = "arn:aws:elasticviews:us-east-1:507922848584:view/favoritemovies-view-Gzwyrxov5Xk4aAY7Gyl3pu"

### Activate the Elastic Views view

In [24]:
!aws elasticviews start-view \
    --view-identifier {viewArn} | jq . 

{
  "view": {
    "viewName": "favoritemovies-view",
    "viewArn": "arn:aws:elasticviews:us-east-1:507922848584:view/favoritemovies-view-Gzwyrxov5Xk4aAY7Gyl3pu",
    "viewDefinition": "SELECT CAST(release_year AS int4), title FROM evfavoritemovies.favoritemovies",
    "provenances": [
      "release_year",
      "title"
    ],
    "viewStatus": "ACTIVE",
    "sourceOrViewArns": [
      "arn:aws:elasticviews:us-east-1:507922848584:source/favoritemovies-1yQgYQbwge35SHdA1yrzTq"
    ],
    "viewSchema": {
      "attributeSchemaList": [
        {
          "attributeName": "release_year",
          "typeDefinition": {
            "typeName": "int",
            "typeConstraints": [
              {
                "constraintName": "ValidValues",
                "constraintValue": "[-2147483648, 2147483647]"
              }
            ]
          }
        },
        {
          "attributeName": "title",
          "typeDefinition": {
            "typeName": "ddb:string",
            "typeCo

### Check the status of the Elastic Views view

In [25]:
!aws elasticviews describe-view \
    --view-identifier {viewArn} | jq . 

{
  "view": {
    "viewName": "favoritemovies-view",
    "viewArn": "arn:aws:elasticviews:us-east-1:507922848584:view/favoritemovies-view-Gzwyrxov5Xk4aAY7Gyl3pu",
    "viewDefinition": "SELECT CAST(release_year AS int4), title FROM evfavoritemovies.favoritemovies",
    "provenances": [
      "release_year",
      "title"
    ],
    "viewStatus": "ACTIVE",
    "sourceOrViewArns": [
      "arn:aws:elasticviews:us-east-1:507922848584:source/favoritemovies-1yQgYQbwge35SHdA1yrzTq"
    ],
    "viewSchema": {
      "attributeSchemaList": [
        {
          "attributeName": "release_year",
          "typeDefinition": {
            "typeName": "int",
            "typeConstraints": [
              {
                "constraintName": "ValidValues",
                "constraintValue": "[-2147483648, 2147483647]"
              }
            ]
          }
        },
        {
          "attributeName": "title",
          "typeDefinition": {
            "typeName": "ddb:string",
            "typeCo

<img src="img/img5.png">

# Step 3: Create a `Redshift` cluster (`TARGET`)

- We have already created a [Redshift Cluster](https://console.aws.amazon.com/redshiftv2/home?region=us-east-1#cluster-details?cluster=redshift-cluster-2) (Let's check via the UI console)

# Step 4: Materialization in Amazon Redshift

Lets open back to `DataGrip` and connect to our `Redshift` cluster

In [26]:
print(viewArn) # We need this while we `CREATE MATERIALIZED VIEW`

arn:aws:elasticviews:us-east-1:507922848584:view/favoritemovies-view-Gzwyrxov5Xk4aAY7Gyl3pu


<img src="img/img6.png">

### Put some new `item` in the Table: 

In [27]:
!aws dynamodb put-item \
    --table-name favoritemovies  \
    --item \
        '{"title": {"S": "DDLJ"}, "release_year": {"N": "1995"}}'

### Run `scan` to check that the data is added to the table 


In [28]:
!aws dynamodb scan --table-name favoritemovies |jq . 


{
  "Items": [
    {
      "release_year": {
        "N": "2008"
      },
      "title": {
        "S": "The Dark Knight"
      }
    },
    {
      "release_year": {
        "N": "2010"
      },
      "title": {
        "S": "Inception"
      }
    },
    {
      "release_year": {
        "N": "1995"
      },
      "title": {
        "S": "DDLJ"
      }
    },
    {
      "release_year": {
        "N": "2016"
      },
      "title": {
        "S": "Moonlight"
      }
    }
  ],
  "Count": 4,
  "ScannedCount": 4,
  "ConsumedCapacity": null
}


# Step 5: Query Amazon `Redshift`

- Lets go back to `DataGrip` and run some query

# Cleanup

- From `Redshift` Cluster 

    - DROP MATERIALIZED VIEW favoritemovies_mv;

    - DROP SCHEMA ev_schema
    

- From `ElasticView` UI

    - Deactivate View

    - Delete View

    - Deactivate Table(source)

    - Delete Table(source)


- From `DynamoDB` UI

    - Delete the table